In [ ]:
#@title Huggingface Login
#@markdown huggingface weight 를 이용하고 싶다면 로그인 필수
from google.colab import userdata
import os

os.environ['HF_WRITE_TOKEN'] = userdata.get('HF_WRITE_TOKEN')

!huggingface-cli login --add-to-git-credential --token $HF_WRITE_TOKEN


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
%%writefile requirements.txt
trl
deepspeed
lightning
datasets
tokenizers


Writing requirements.txt


In [ ]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

Cloning into 'mergekit'...
remote: Enumerating objects: 2157, done.
remote: Counting objects: 100% (472/472), done.
remote: Compressing objects: 100% (209/209), done.
remote: Total 2157 (delta 318), reused 378 (delta 263), pack-reused 1685 (from 1)
Receiving objects: 100% (2157/2157), 668.50 KiB | 3.59 MiB/s, done.
Resolving deltas: 100% (1462/1462), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 959.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import yaml

MODEL_NAME = "Marcoro14-7B-slerp"
yaml_config = """
slices:
  - sources:
      - model: andrijdavid/Marcoroni-7B-v3-GGUF
        layer_range: [0, 32]
      - model: EmbeddedLLM/Mistral-7B-Merge-14-v0.1
        layer_range: [0, 32]
merge_method: slerp
base_model: andrijdavid/Marcoroni-7B-v3-GGUF
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)


In [ ]:
# Merge models
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle


config.json: 100% 604/604 [00:00<00:00, 3.36MB/s]
Warmup loader cache:   0% 0/2 [00:00<?, ?it/s]
Fetching 8 files:   0% 0/8 [00:00<?, ?it/s]

added_tokens.json: 100% 42.0/42.0 [00:00<00:00, 196kB/s]

Fetching 8 files:  12% 1/8 [00:00<00:01,  5.67it/s]

pytorch_model.bin.index.json: 100% 23.9k/23.9k [00:00<00:00, 18.7MB/s]


special_tokens_map.json: 100% 145/145 [00:00<00:00, 653kB/s]


tokenizer_config.json: 100% 953/953 [00:00<00:00, 5.77MB/s]


tokenizer.json:   0% 0.00/1.80M [00:00<?, ?B/s]


generation_config.json: 100% 111/111 [00:00<00:00, 429kB/s]


tokenizer.json: 100% 1.80M/1.80M [00:00<00:00, 12.0MB/s]

Fetching 8 files:  75% 6/8 [00:00<00:00, 19.10it/s]

tokenizer.model: 100% 493k/493k [00:00<00:00, 8.14MB/s]
Fetching 8 files: 100% 8/8 [00:00<00:00, 17.83it/s]
Warmup loader cache:  50% 1/2 [00:00<00:00,  1.58it/s]
Fetching 8 files:   0% 0/8 [00:00<?, ?it/s]

model.safetensors.index.json:   0% 0.00/22.8k [00:00<?, ?B/s]


special_tokens_map.json:   0% 0.00/414 [00:00<?, ?B/s]